In [9]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.stats import pearsonr
import seaborn

In [15]:
dumpAB = torch.load('gradinfo_AB.pth')
dumpBH = torch.load('gradinfo_BH.pth')
grad_sims = dumpAB['grad_sims']
grad_sims1 = dumpBH['grad_sims']

In [38]:
#AB
acc_AB_A = [1.915,5.426,10.106,13.085,19.149,20.319,19.894,23.723,26.170,24.787]
acc_AB_B = [5.591,19.441,28.463,29.352,35.070,31.258,36.595,32.656,36.595,36.341]

#BH
acc_BH_B = [8.259,29.860,36.849,37.357,41.550,39.517,43.456,42.058,42.313,45.743]
acc_BH_H = [8.418,34.056,38.903,45.281,45.153,45.918,47.449,48.214,48.214,47.959]

In [45]:
p_arabacc = []
p_banacc_AB = []
i=0
print('gradient cosine and accuracy Correlation scores for ARAB-BAN\n')
for name,sims in grad_sims.items():
    p_arabacc.append(pearsonr(np.array(acc_AB_A),np.array(sims))[0])
    p_banacc_AB.append(pearsonr(np.array(acc_AB_B),np.array(sims))[0])
    print(name)
    print('correlation between gradient similarity and arab accuracy:',p_arabacc[i])
    print('correlation between gradient similarity and ban accuracy:',p_banacc_AB[i])
    print()
    i+=1

gradient cosine and accuracy Correlation scores for ARAB-BAN

FeatureExtraction.ConvNet.0.weight
correlation between gradient similarity and arab accuracy: 0.2013695414894921
correlation between gradient similarity and ban accuracy: 0.20370367841910603

FeatureExtraction.ConvNet.0.bias
correlation between gradient similarity and arab accuracy: -0.1220398546818001
correlation between gradient similarity and ban accuracy: -0.020925813986085316

FeatureExtraction.ConvNet.3.weight
correlation between gradient similarity and arab accuracy: -0.04744393695821054
correlation between gradient similarity and ban accuracy: -0.013492182821065372

FeatureExtraction.ConvNet.3.bias
correlation between gradient similarity and arab accuracy: 0.11201980192492327
correlation between gradient similarity and ban accuracy: 0.3146812925081734

FeatureExtraction.ConvNet.6.weight
correlation between gradient similarity and arab accuracy: 0.3991847153166311
correlation between gradient similarity and ban accura

In [43]:
p_hinacc = []
p_banacc = []
i = 0
print('gradient cosine and accuracy Correlation scores for Hin-BAN\n')
for name,sims in grad_sims1.items():
    p_hinacc.append(pearsonr(np.array(acc_BH_H),np.array(sims))[0])
    p_banacc.append(pearsonr(np.array(acc_BH_B),np.array(sims))[0])
    print(name)
    print('correlation between gradient similarity and hindi accuracy:',p_hinacc[i])
    print('correlation between gradient similarity and ban accuracy:',p_banacc[i])
    i = i+1
    print()

gradient cosine and accuracy Correlation scores for Hin-BAN

FeatureExtraction.ConvNet.0.weight
correlation between gradient similarity and hindi accuracy: 0.5358637825030929
correlation between gradient similarity and ban accuracy: 0.5785184269851343

FeatureExtraction.ConvNet.0.bias
correlation between gradient similarity and hindi accuracy: -0.23522956641075712
correlation between gradient similarity and ban accuracy: -0.32659319451954044

FeatureExtraction.ConvNet.3.weight
correlation between gradient similarity and hindi accuracy: 0.6739424729678521
correlation between gradient similarity and ban accuracy: 0.6076886119101637

FeatureExtraction.ConvNet.3.bias
correlation between gradient similarity and hindi accuracy: 0.05988170006593138
correlation between gradient similarity and ban accuracy: 0.005724560266620152

FeatureExtraction.ConvNet.6.weight
correlation between gradient similarity and hindi accuracy: 0.8691076574466235
correlation between gradient similarity and ban accura

In [49]:
#compute euclidean distance between correlation vectors
def norm1(a,b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a-b)

In [55]:
pearson_norm_AB = norm1(p_arabacc,p_banacc_AB)
pearson_norm_BH = norm1(p_hinacc,p_banacc)
pearson_norm_BH_trainAB = norm1(p_hinacc,p_arabacc)
print('norm between pearson correlation vectors of arab and bangla [training on Arab-Bangla]:',pearson_norm_AB)
print('norm between pearson correlation vectors of hindi and bangla [training on Hindi-Bangla]:',pearson_norm_BH)
print('norm between pearson correlation vectors of hindi and bangla not trained together:',pearson_norm_BH_trainAB)

norm between pearson correlation vectors of arab and bangla [training on Arab-Bangla]: 1.1186875548754143
norm between pearson correlation vectors of hindi and bangla [training on Hindi-Bangla]: 0.19313257876804363
norm between pearson correlation vectors of hindi and bangla not trained together: 2.7278657037466605


In [27]:
sum(p_arabacc)/len(p_arabacc)

0.23100178569879157

In [28]:
sum(p_banacc)/len(p_banacc)

0.3780566580236156

In [31]:
pearsonr(np.array(p_hinacc),np.array(p_banacc))

(0.9956100192364844, 1.4386756157777825e-36)

In [23]:
sum(p_hinacc)/len(p_hinacc)

0.5924139804953318

In [24]:
sum(p_banacc)/len(p_banacc)

0.5873428894708633